In [1]:
import h5py
import healpy as hp
import numpy as np
import os

In [5]:
!ls -lsh /project/chihway/dhayaa/DECADE/Foreground_Masks/*

385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:07 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.05_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:04 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.1_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:19 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star10.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:22 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star20.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:10 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star3.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:13 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star4.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 10:16 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits
385M -rw-rw-r-- 1 dhayaa pi-chihway 385M Nov  5 09:58 /project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.3_Star5.fi

In [6]:
!ls /scratch/midway3/dhayaa/SHEARTESTS/

In [7]:
PATHS = ['/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.4_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.3_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.1_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.05_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star3.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star4.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star10.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star20.fits']

NAMES = ['Ext0.4_Star5',
         'Ext0.3_Star5',
         'Ext0.2_Star5',
         'Ext0.1_Star5',
         'Ext0.05_Star5',
         'Ext0.2_Star3',
         'Ext0.2_Star4',
         'Ext0.2_Star5',
         'Ext0.2_Star10',
         'Ext0.2_Star20']

for NAME, PATH in zip(NAMES, PATHS):
    
    GOLD_Foreground  = hp.read_map(PATH, dtype = int)
        
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)
    
    with h5py.File('/project/chihway/data/decade/metacal_gold_combined_20231002.hdf', 'r') as f:
        M = ((f['sg_bdf'][:] >= 4) & (f['mcal_s2n_noshear'][:] > 3) & 
             (f['mcal_T_ratio_noshear'][:] > 0.2) & (f['mcal_flags'][:] == 0))

        FLAGS_Foreground = GOLD_Foreground[hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['RA'][:], f['DEC'][:], lonlat = True)]

        M = M & (FLAGS_Foreground ==  0)
        np.save(path + '/shear_inds.npy', np.where(M)[0])


    with h5py.File('/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5', 'r') as f:
        M = (f['FLUX_APER_8'][:]/f['FLUXERR_APER_8'][:] > 1)

        FLAGS_Foreground = GOLD_Foreground[hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['ra'][:], f['dec'][:], lonlat = True)]

        M = M & (FLAGS_Foreground ==  0)
        np.save(path + '/psf_inds.npy', np.where(M)[0])
        
        
    JOB_TXT = """#!/bin/bash
#SBATCH --job-name ShearTests_%(NAME)s
#SBATCH --partition=caslake
#SBATCH --account=pi-chihway
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=48
#SBATCH --time=30:00:00
#SBATCH --output=/home/dhayaa/DECADE/shearcat/shear_tests/runs/%(NAME)s.log
#SBATCH --mail-user=dhayaa@uchicago.edu
#SBATCH --mail-type=BEGIN,END

#X=${TMPDIR}
#echo $TMPDIR
if [ "$USER" == "dhayaa" ]
then
    conda activate /project/chihway/dhayaa/MyEnvs_Midway3/shear
fi

shearcatalog=/project/chihway/data/decade/metacal_gold_combined_20230613.hdf
psfcatalog=/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5

RUN_DIR=/home/dhayaa/DECADE/shearcat/shear_tests

python -u $RUN_DIR/ShearTestRunner.py --psf_cat "/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5" \\
                                      --galaxy_cat "/project/chihway/data/decade/metacal_gold_combined_20231002.hdf" \\
                                      --psf_cat_inds "%(PSF_INDS)s" \\
                                      --galaxy_cat_inds "%(SHEAR_INDS)s" \\
                                      --output_path "%(OUTPUT)s" \\
                                      --sim_Cls_path "/project/chihway/dhayaa/DECADE/cosmosis/Lucas_files/Kappa_Cls.txt"

    """
    
    args = {'NAME' : NAME,
            'PSF_INDS' : path + '/psf_inds.npy',
            'SHEAR_INDS' : path + '/shear_inds.npy',
            'OUTPUT' : path}
    
    with open('runs/job_%s.sh' % NAME, 'w') as f:
        
        f.write(JOB_TXT % args)

    print("DONE WITH", NAME)

/home/dhayaa/ipykernel_754720/2003582104.py:41: RuntimeWarning: invalid value encountered in true_divide
  M = (f['FLUX_APER_8'][:]/f['FLUXERR_APER_8'][:] > 1)


DONE WITH Ext0.4_Star5
DONE WITH Ext0.3_Star5
DONE WITH Ext0.2_Star5
DONE WITH Ext0.1_Star5
DONE WITH Ext0.05_Star5
DONE WITH Ext0.2_Star3
DONE WITH Ext0.2_Star4


mkdir: cannot create directory ‘/scratch/midway3/dhayaa/SHEARTESTS/Ext0.2_Star5’: File exists


DONE WITH Ext0.2_Star5
DONE WITH Ext0.2_Star10
DONE WITH Ext0.2_Star20


In [3]:
PATHS = ['/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.4_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.3_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.1_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.05_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star3.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star4.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star5.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star10.fits',
         '/project/chihway/dhayaa/DECADE/Foreground_Masks/GOLD_Ext0.2_Star20.fits']

NAMES = ['Ext0.4_Star5',
         'Ext0.3_Star5',
         'Ext0.2_Star5',
         'Ext0.1_Star5',
         'Ext0.05_Star5',
         'Ext0.2_Star3',
         'Ext0.2_Star4',
         'Ext0.2_Star5',
         'Ext0.2_Star10',
         'Ext0.2_Star20']

for NAME, PATH in zip(NAMES, PATHS):
    
    GOLD_Foreground  = hp.read_map(PATH, dtype = int)
        
    path = '/scratch/midway3/dhayaa/SHEARTESTS/%s' % NAME
    os.system('mkdir %s' % path)
    
    with h5py.File('/project/chihway/data/decade/metacal_gold_combined_20231002.hdf', 'r') as f:
        M = ((f['sg_bdf'][:] >= 4) & (f['mcal_s2n_noshear'][:] > 3) & 
             (f['mcal_T_ratio_noshear'][:] > 0.2) & (f['mcal_flags'][:] == 0))

        FLAGS_Foreground = GOLD_Foreground[hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['RA'][:], f['DEC'][:], lonlat = True)]

        M = M & (FLAGS_Foreground ==  0)
        np.save(path + '/shear_inds.npy', np.where(M)[0])


    with h5py.File('/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5', 'r') as f:
        M = (f['FLUX_APER_8'][:]/f['FLUXERR_APER_8'][:] > 1)

        FLAGS_Foreground = GOLD_Foreground[hp.ang2pix(hp.npix2nside(GOLD_Foreground.size), f['ra'][:], f['dec'][:], lonlat = True)]

        M = M & (FLAGS_Foreground ==  0)
        np.save(path + '/psf_inds.npy', np.where(M)[0])
        
        
    JOB_TXT = """#!/bin/bash
#SBATCH --job-name ShearTests_%(NAME)s
#SBATCH --partition=kicp
#SBATCH --account=kicp
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=48
#SBATCH --time=30:00:00
#SBATCH --output=/home/dhayaa/DECADE/shearcat/shear_tests/runs/%(NAME)s.log
#SBATCH --mail-user=dhayaa@uchicago.edu
#SBATCH --mail-type=BEGIN,END

#X=${TMPDIR}
#echo $TMPDIR
if [ "$USER" == "dhayaa" ]
then
    source /home/dhayaa/DECADE/mcal_sim_test/bash_profile.sh
    conda activate /project/chihway/dhayaa/MyEnvs_Midway3/shear
fi

shearcatalog=/project/chihway/data/decade/metacal_gold_combined_20230613.hdf
psfcatalog=/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5

RUN_DIR=/home/dhayaa/DECADE/shearcat/shear_tests

python -u $RUN_DIR/ShearTestRunner.py --psf_cat "/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5" \\
                                      --galaxy_cat "/project/chihway/data/decade/metacal_gold_combined_20231002.hdf" \\
                                      --psf_cat_inds "%(PSF_INDS)s" \\
                                      --galaxy_cat_inds "%(SHEAR_INDS)s" \\
                                      --output_path "%(OUTPUT)s" \\
                                      --sim_Cls_path "/project/chihway/dhayaa/DECADE/cosmosis/Lucas_files/Kappa_Cls.txt"

    """
    
    args = {'NAME' : NAME,
            'PSF_INDS' : path + '/psf_inds.npy',
            'SHEAR_INDS' : path + '/shear_inds.npy',
            'OUTPUT' : path}
    
    with open('runs/job_%s.sh' % NAME, 'w') as f:
        
        f.write(JOB_TXT % args)

    print("DONE WITH", NAME)

['BDF_FLUX_ERR_G', 'BDF_FLUX_ERR_I', 'BDF_FLUX_ERR_R', 'BDF_FLUX_ERR_Z', 'BDF_FLUX_G', 'BDF_FLUX_I', 'BDF_FLUX_R', 'BDF_FLUX_Z', 'BDF_S2N', 'BDF_T', 'COADD_OBJECT_ID', 'DEC', 'FLAGS_FOREGROUND', 'FLUXERR_AUTO_G', 'FLUXERR_AUTO_I', 'FLUXERR_AUTO_R', 'FLUXERR_AUTO_Z', 'FLUX_AUTO_G', 'FLUX_AUTO_I', 'FLUX_AUTO_R', 'FLUX_AUTO_Z', 'FLUX_RADIUS_G', 'FLUX_RADIUS_I', 'FLUX_RADIUS_R', 'FLUX_RADIUS_Z', 'Ncutouts_raw', 'RA', 'badfrac', 'id', 'mcal_T_1m', 'mcal_T_1p', 'mcal_T_2m', 'mcal_T_2p', 'mcal_T_noshear', 'mcal_T_ratio_1m', 'mcal_T_ratio_1p', 'mcal_T_ratio_2m', 'mcal_T_ratio_2p', 'mcal_T_ratio_noshear', 'mcal_flags', 'mcal_flux_1m', 'mcal_flux_1p', 'mcal_flux_2m', 'mcal_flux_2p', 'mcal_flux_err_1m', 'mcal_flux_err_1p', 'mcal_flux_err_2m', 'mcal_flux_err_2p', 'mcal_flux_err_noshear', 'mcal_flux_noshear', 'mcal_g_1m', 'mcal_g_1p', 'mcal_g_2m', 'mcal_g_2p', 'mcal_g_cov_1m', 'mcal_g_cov_1p', 'mcal_g_cov_2m', 'mcal_g_cov_2p', 'mcal_g_cov_noshear', 'mcal_g_noshear', 'mcal_g_w', 'mcal_g_w_v2', 'mcal

/home/dhayaa/ipykernel_1931160/2300199348.py:15: RuntimeWarning: invalid value encountered in true_divide
  M = (f['FLUX_APER_8'][:]/f['FLUXERR_APER_8'][:] > 40)


In [5]:
np.load('/scratch/midway3/dhayaa/SHEARTEST/shear_inds.npy').shape

(111120479,)

In [15]:
np.load('/scratch/midway3/dhayaa/SHEARTEST/shear_inds.npy').shape

(168485318,)

In [5]:
with h5py.File('/project/chihway/data/decade/metacal_gold_combined_20231002.hdf', 'r') as f:
    
    print(f['RA'].shape)

(597650947,)


In [6]:
np.load('/scratch/midway3/dhayaa/SHEARTEST/psf_inds.npy').shape

(182147549,)

In [16]:
np.load('/scratch/midway3/dhayaa/SHEARTEST/psf_inds.npy').shape

(354987624,)

In [18]:
with h5py.File('/project/chihway/dhayaa/DECADE/star_psf_shapecatalog_20230510.hdf5', 'r') as f:
    
    print(f['ra'].shape)

(721355058,)


In [6]:
import treecorr

In [ ]:
X

In [13]:
X = treecorr.Catalog(x = np.arange(100), y = np.arange(100)* - 1, npatch = 5)
Y = treecorr.Catalog(x = np.arange(1000), y = np.arange(1000)* - 1, patch_centers=X._centers)

In [15]:
vars(Y)

{'config': {'comment_marker': '#',
  'first_row': 1,
  'last_row': -1,
  'every_nth': 1,
  'x_col': '0',
  'y_col': '0',
  'z_col': '0',
  'ra_col': '0',
  'dec_col': '0',
  'r_col': '0',
  'g1_col': '0',
  'g2_col': '0',
  'k_col': '0',
  'patch_col': '0',
  'w_col': '0',
  'wpos_col': '0',
  'flag_col': '0',
  'ok_flag': 0,
  'allow_xyz': False,
  'flip_g1': False,
  'flip_g2': False,
  'keep_zero_weight': False,
  'npatch': 1,
  'kmeans_init': 'tree',
  'kmeans_alt': False,
  'verbose': 1,
  'split_method': 'mean',
  'cat_precision': 16},
 'orig_config': {},
 '_num': 0,
 '_is_rand': False,
 'logger': <Logger treecorr (WARNING)>,
 '_x': array([  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.,  10.,
         11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.,  20.,  21.,
         22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.,  30.,  31.,  32.,
         33.,  34.,  35.,  36.,  37.,  38.,  39.,  40.,  41.,  42.,  43.,
         44.,  45.,  46.,  47.,  48.,  49.,  50.,  51.,  5

In [5]:
np.savetxt('/scratch/midway3/dhayaa/SHEARTEST/Cls.npy', np.zeros(3*1024 - 1))